In [5]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [2]:
# Set the path to the directory containing the subdirectories
root_dir = 'Dataset'

In [9]:
# Set the number of frames to use per video
num_frames = 30

# Set the size of the input frames
input_shape = (224, 224)

# Set the number of classes
num_classes = 2

# Set the batch size
batch_size = 32

# Set the number of epochs
epochs = 10

In [6]:
# Define the CNN architecture
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(num_frames, input_shape[0], input_shape[1], 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(256, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(256))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [7]:
# Define the data generator for data augmentation
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [10]:
# Loop through each subdirectory in the root directory
for subdir, dirs, files in os.walk(root_dir):
    # Load the video files in the subdirectory
    videos = []
    for file in files:
        if file.endswith('.mp4') or file.endswith('.avi'):
            video_path = os.path.join(subdir, file)
            cap = cv2.VideoCapture(video_path)
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, input_shape)
                frames.append(frame)
            cap.release()
            frames = np.array(frames)
            if len(frames) >= num_frames:
                videos.append(frames[:num_frames])

    # Prepare the input and target data
    if len(videos) > 0:
        X = np.array(videos)
        y = np.zeros((X.shape[0], num_classes))
        y[:, int(subdir[-1]) - 1] = 1

        # Apply data augmentation
        X = np.concatenate([data_generator.random_transform(x) for x in X], axis=0)

        # Train the model
        model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[early_stop])

        model.save('sign-language.h5')